## Imports

In [ ]:
import rdflib
from rdflib import RDF, RDFS, XSD, OWL, URIRef
from rdflib.paths import OneOrMore
from bmo_tools.ontologies import subontology_from_term

## Extract UBERON subontology (top down from 'regional part of brain')

In [ ]:
g = rdflib.Graph()
g.parse("/Users/akkaufma/Desktop/uberon.owl")

Find the term of interest

In [ ]:
label = "regional part of brain"
term_to_expand = ""
for s in g.subjects(RDFS.label, rdflib.Literal(label)):
    term_to_expand = s
    break

In [ ]:
term_to_expand

In [ ]:
uberon = subontology_from_term(g, term_to_expand, closed=True)

## Load MBA ontology

In [ ]:
mba = rdflib.Graph()
mba.parse("../../ontologies/external/allen_MBA_ontology_ccfv3.ttl", format="turtle")

## New properties

In [ ]:
delineates_rel = rdflib.URIRef("https://bbp.epfl.ch/ontologies/core/bmo/delineates")
delineatedby_rel = rdflib.URIRef("https://bbp.epfl.ch/ontologies/core/bmo/delineatedBy")

## Add new triples to onotlogy: If MBA term present, add delineatedBy relationship

In [ ]:
for s, p, o in uberon.triples((None, URIRef("http://www.geneontology.org/formats/oboInOwl#hasDbXref"), None)):
    if str(o).startswith("MBA"):
        mba_id = URIRef(f"http://api.brain-map.org/api/v2/data/Structure/{o.split(':')[-1]}")
        print(f"{s} {p} {o}")
        uberon.add((s, delineatedby_rel, mba_id))
        mba.add((mba_id, delineates_rel, s))

In [ ]:
brainontology = uberon

In [ ]:
brainontology.serialize(destination="/Users/akkaufma/Desktop/brain-region-ontology.ttl")